In [21]:
%load_ext autoreload
%autoreload 2

import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords

import pandas as pd
import scipy
from sklearn import *
import re
from SimpleCountVectorizer_MAD import *
from TFIDFVectorizer import *

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.grammar import ProbabilisticProduction

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
"""Given an int32 number, print it in English."""
def int_to_en(num):
    d = { 0 : 'zero', 1 : 'one', 2 : 'two', 3 : 'three', 4 : 'four', 5 : 'five',
          6 : 'six', 7 : 'seven', 8 : 'eight', 9 : 'nine', 10 : 'ten',
          11 : 'eleven', 12 : 'twelve', 13 : 'thirteen', 14 : 'fourteen',
          15 : 'fifteen', 16 : 'sixteen', 17 : 'seventeen', 18 : 'eighteen',
          19 : 'nineteen', 20 : 'twenty',
          30 : 'thirty', 40 : 'forty', 50 : 'fifty', 60 : 'sixty',
          70 : 'seventy', 80 : 'eighty', 90 : 'ninety' }
    k = 1000
    m = k * 1000
    b = m * 1000
    t = b * 1000

    assert(0 <= num)

    if (num < 20):
        return d[num]

    if (num < 100):
        if num % 10 == 0: return d[num]
        else: return d[num // 10 * 10] + '-' + d[num % 10]

    if (num < k):
        if num % 100 == 0: return d[num // 100] + ' hundred'
        else: return d[num // 100] + ' hundred and ' + int_to_en(num % 100)

    if (num < m):
        if num % k == 0: return int_to_en(num // k) + ' thousand'
        else: return int_to_en(num // k) + ' thousand, ' + int_to_en(num % k)

    if (num < b):
        if (num % m) == 0: return int_to_en(num // m) + ' million'
        else: return int_to_en(num // m) + ' million, ' + int_to_en(num % m)

    if (num < t):
        if (num % b) == 0: return int_to_en(num // b) + ' billion'
        else: return int_to_en(num // b) + ' billion, ' + int_to_en(num % b)

    if (num % t == 0): return int_to_en(num // t) + ' trillion'
    else: return int_to_en(num // t) + ' trillion, ' + int_to_en(num % t)

    raise AssertionError('num is too large: %s' % str(num))

# Count Vectorizer

In [3]:
train_df = pd.read_csv("./input/quora_train_data.csv")
test_df = pd.read_csv('./input/quora_test_data.csv')

# train_df, test_df = sklearn.model_selection.train_test_split(train_df, test_size=0.1, random_state=123)

In [4]:
train_df.shape, test_df.shape

((323432, 6), (80858, 6))

In [5]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    
    # return list(map(lambda x: str(x), all_questions)) # Slower
    return [str(s) for s in mylist]

In [6]:
all_questions = cast_list_as_strings(list(train_df.loc[:, 'question1'])+list(train_df.loc[:, 'question2']))
print(set(type(x).__name__ for x in all_questions))

{'str'}


In [14]:
def num_conv(s):
    try:
        return int_to_en(int(s)).replace(",","").replace(" ","_")
    except:
        return s


def my_doc_cleaner(doc,
                  pat=r"[^a-zA-Z0-9]"):
    # Allow alphanumeric characters
    doc_cleaner_pattern=pat
    clean_doc_pattern = re.compile(doc_cleaner_pattern)
    doc_clean = clean_doc_pattern.sub(" ", doc)
    return doc.lower()


# stpw = set(stopwords.words("english"))
stpw = []

stemmer =  SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

def my_tokenizer_func(doc, 
                      ngrams=(1,3), 
                      numbers_to_words=True,
                      stop_words=stpw,
                      pat=r"(?u)\b\w\S*\w*\b",
                      lem=True,
                      stem=True,
                      n_tokens=True,
                     duplicate_question_words=True):
    import pdb
    # Split using a patterm
#     pat=r"(?u)\b\w\w+\b"
#     pat=r"(?u)\b\w\S*\w*\b"
    
    token_pattern = re.compile(pat)
    lst = token_pattern.findall(doc)
    
    # Duplicate key_words
    if duplicate_question_words:
        question_words = ['who','what','when','where','why','how']
        lst += [value for value in lst if value in question_words] 
        
    # Transform numbers into words
    if numbers_to_words:
        lst = list(map(lambda x: num_conv(x), lst))
        
    # Drop stopwords 
    lst = list(filter(lambda x : x not in stop_words, lst))
    
    #Stemmer
    if stem:
        lst = list(map(lambda x: stemmer.stem(x), lst))
    
    #Lemmatizer 
    if lem:
        lst = list(map(lambda x: lemmatizer.lemmatize(x), lst))
     

    
    if ngrams==(1,1):
        return lst
    
    # Generate ngrams
    lstRet = []
    for a in range(ngrams[0], ngrams[1]+1):
        if a!=1:
            lstRet+=list(zip(*[lst[i:] for i in range(a)]))
    

    
    # N-tokens
    if n_tokens:
        lst.append(num_conv(str(len(lst))) + 'tokens')
        
    
    
    return lstRet if ngrams[0]!=1 else lst+lstRet

In [15]:
count_vect = SimpleCountVectorizer(
    doc_cleaner_func=my_doc_cleaner,
    tokenizer_func=my_tokenizer_func
)
count_vect.fit(all_questions)

SimpleCountVectorizer(doc_cleaner_func=<function my_doc_cleaner at 0x000001CFA685A438>,
                      doc_cleaner_pattern='[^a-zA-Z]',
                      dtype=<class 'numpy.float32'>, min_word_counts=1,
                      token_pattern='(?u)\\b\\w\\w+\\b',
                      tokenizer_func=<function my_tokenizer_func at 0x000001CFC6692288>,
                      word_transformer_func=None)

In [16]:
def get_features_from_df(df, vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    q1 = vectorizer.transform(q1_casted)
    q2 = vectorizer.transform(q2_casted)
    
    X_q1q2 = scipy.sparse.hstack((q1,q2))
        
    return X_q1q2

In [17]:
X_tr_q1q2 = get_features_from_df(train_df,count_vect)
X_te_q1q2  = get_features_from_df(test_df, count_vect)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

['who', 'what', 'when', 'where', 'why', 'how']
['who', 'what', 'when', 'where', 'why', 'how']
['who', 'what', 'when', 'where', 'why', 'how']
['who', 'what', 'when', 'where', 'why', 'how']


((323432, 6675538), (323432, 6), (80858, 6), (80858, 6675538))

In [18]:
y_train = train_df["is_duplicate"].values
y_test = test_df['is_duplicate'].values

## Linear Regression

In [19]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear", verbose=1, max_iter=100)
logistic.fit(X_tr_q1q2, y_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [14]:
logistic.score(X_tr_q1q2, y_train), logistic.score(X_te_q1q2, y_test)
# original

(0.9963639961413837, 0.8085285314996661)

In [23]:
logistic.score(X_tr_q1q2, y_train), logistic.score(X_te_q1q2, y_test)
# original

(0.9978388038289346, 0.8110267382324569)

In [54]:
logistic.score(X_tr_q1q2, y_train), logistic.score(X_te_q1q2, y_test)
# added n_token

(0.9963083430210987, 0.8101362883078977)

In [13]:
logistic.score(X_tr_q1q2, y_train), logistic.score(X_te_q1q2, y_test)
# n_tokens and duplicate_question_word

(0.9977305894283807, 0.8116945756758762)

In [20]:
logistic.score(X_tr_q1q2, y_train), logistic.score(X_te_q1q2, y_test)
# n_tokens and modified duplicate_question_word (triple question word)

(0.9978388038289346, 0.8110267382324569)

## XGBoost

In [42]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# PARAM GRID
param_grid = {}
param_grid['objective'] = 'binary:logistic'
param_grid['eval_metric'] = ['auc', 'logloss']   #error: (wrong cases)/#(all cases)
param_grid['eta'] = 0.02
param_grid['max_depth'] = 10
param_grid['n_estimators'] = 300

d_train = xgb.DMatrix(X_tr_q1q2, label=y_train)  # For sparse matrices
d_valid = xgb.DMatrix(X_te_q1q2, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(param_grid, d_train, 300, watchlist, early_stopping_rounds=50, verbose_eval=25, )
#300 # train-auc:0.818362	train-logloss:0.527855	valid-auc:0.801083	valid-logloss:0.53815

[0]	train-auc:0.60892	train-logloss:0.69073	valid-auc:0.60823	valid-logloss:0.69080
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[25]	train-auc:0.60901	train-logloss:0.66507	valid-auc:0.60830	valid-logloss:0.66550
[50]	train-auc:0.60901	train-logloss:0.65734	valid-auc:0.60830	valid-logloss:0.65794
[75]	train-auc:0.60901	train-logloss:0.65435	valid-auc:0.60830	valid-logloss:0.65522
[100]	train-auc:0.60901	train-logloss:0.65327	valid-auc:0.60830	valid-logloss:0.65428


In [24]:
def get_mistakes(clf, X_q1q2, y):

    predictions = clf.predict(X_q1q2)    
    incorrect_predictions = predictions!=y

    incorrect_indices = np.where(incorrect_predictions)[0]
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions
    
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])
    
def print_mistake_k_and_tokens(k, mistake_indices, predictions,
                               X_q1q2, count_vect, clf):
    q1 = train_df.iloc[mistake_indices[k]].question1
    q2 = train_df.iloc[mistake_indices[k]].question2
    
    print(q1)
    print(count_vect.tokenize(q1))
    print()
    print(q2)
    print(count_vect.tokenize(q2))
    print()
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])
    print()
    print("Probability vector: [P(0|x), P(1|x)]:")
    print(clf.predict_proba(X_q1q2)[mistake_indices[k],:])
    
    
def hist_errors(mistake_indices, predictions,
                               X_q1q2, count_vect, clf):
    qs = train_df.iloc[mistake_indices][['question1', 'question2']]
    qs['true_class']=train_df.iloc[mistake_indices].is_duplicate
    qs['prediction']=predictions[mistake_indices]
    qs['P(1|x)']=clf.predict_proba(X_q1q2)[mistake_indices,:][:,1]
    qs = qs.reset_index(drop=True)
    return qs


In [25]:
mistake_indices, predictions = get_mistakes(logistic, X_te_q1q2, y_test)

In [26]:
print_mistake_k(0, mistake_indices, predictions)

How do I study for Honeywell company recruitment?
How do I study for Honeywell company recruitments?
true class: 1
prediction: 0


In [28]:
print_mistake_k_and_tokens(0, mistake_indices, predictions,
                           X_te_q1q2, count_vect, logistic)

How do I study for Honeywell company recruitment?
['how', 'do', 'i', 'studi', 'for', 'honeywel', 'compani', 'recruit', ('how', 'do'), ('do', 'i'), ('i', 'studi'), ('studi', 'for'), ('for', 'honeywel'), ('honeywel', 'compani'), ('compani', 'recruit'), ('how', 'do', 'i'), ('do', 'i', 'studi'), ('i', 'studi', 'for'), ('studi', 'for', 'honeywel'), ('for', 'honeywel', 'compani'), ('honeywel', 'compani', 'recruit')]

How do I study for Honeywell company recruitments?
['how', 'do', 'i', 'studi', 'for', 'honeywel', 'compani', 'recruit', ('how', 'do'), ('do', 'i'), ('i', 'studi'), ('studi', 'for'), ('for', 'honeywel'), ('honeywel', 'compani'), ('compani', 'recruit'), ('how', 'do', 'i'), ('do', 'i', 'studi'), ('i', 'studi', 'for'), ('studi', 'for', 'honeywel'), ('for', 'honeywel', 'compani'), ('honeywel', 'compani', 'recruit')]

true class: 1
prediction: 0

Probability vector: [P(0|x), P(1|x)]:
[0.81332746 0.18667254]


# TFIDF

In [ ]:
tfidf_vectorizer = TFIDFVectorizer(count_vect.vocabulary, count_vect.word_to_ind, count_vect.tokenize)
tfidf_vectorizer.fit(all_questions)

In [ ]:
X_tfidf_tr_q1q2 = get_features_from_df(train_df, tfidf_vectorizer)
X_tfidf_te_q1q2  = get_features_from_df(test_df, tfidf_vectorizer)

X_tfidf_tr_q1q2.shape, train_df.shape, test_df.shape, X_tfidf_te_q1q2.shape

In [ ]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear", verbose=1, max_iter=100)
logistic.fit(X_tfidf_tr_q1q2, y_train)

In [ ]:
logistic.score(X_tfidf_tr_q1q2, y_train), logistic.score(X_tfidf_te_q1q2, y_test)

In [ ]:
mistake_indices, predictions = get_mistakes(logistic, X_tfidf_te_q1q2, y_test)

In [ ]:
print_mistake_k(0, mistake_indices, predictions)

In [ ]:
print_mistake_k_and_tokens(0, mistake_indices, predictions,
                           X_tfidf_te_q1q2, tfidf_vectorizer, logistic)

In [ ]:
######### SEARCH QUESTIONS WITH SAME TOKENS ###########

q1_casted =  cast_list_as_strings(list(train_df["question1"]))
q2_casted =  cast_list_as_strings(list(train_df["question2"]))

q1 = count_vect.transform(q1_casted)
q2 = count_vect.transform(q2_casted)

same_tokens_idxs = []
for i in range(len(q1_casted)):
    same_features = ( q1[i] != q2[i] ).nnz == 0
    if same_features:
        same_tokens_idxs.append(i)
    if i % 500 == 0: print(i)

same_tokens_idxs = np.array(same_tokens_idxs)


same_tokens = train_df.iloc[same_tokens_idxs]
duplicates = same_tokens['is_duplicate'] == 1

In [ ]:
same_tokens[duplicates]

In [ ]:
same_tokens[~duplicates]